In [1]:
!pip install scikit-learn-intelex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 MB 11.3 MB/s eta 0:00:00


In [2]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


**Imports**

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

**Pre-Processing**

In [ ]:
data = pd.read_csv('../data/WildfireData.csv')
data.head()

In [ ]:
data = data[['fire_size', 'remoteness', 'putout_time', 'stat_cause_descr']]
data.head()

In [ ]:
data = data.dropna()
data['putout_time'] = data['putout_time'].str.split(' ').str[0]
data.head()

In [ ]:
data.stat_cause_descr.unique()

In [ ]:
data['stat_cause_descr'] = data['stat_cause_descr'].map({
    'Debris Burning': 0,
    'Arson': 1,
    'Lightning': 2,
    'Equipment Use': 3,
    'Campfire': 4,
    'Structure': 5,
    'Children': 5,
    'Smoking': 5,
    'Powerline': 5,
    'Railroad': 5,
    'Fireworks': 5,
    'Missing/Undefined': 5,
    'Miscellaneous': 5
})
data.stat_cause_descr.unique()
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop('stat_cause_descr', axis=1).astype('float')
y = data.stat_cause_descr.astype('float')
print(X.head())
print(y.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
 y_test[0]

In [ ]:
import seaborn as sns
sns.heatmap(df.corr(),annot=True)

**NN Model**

In [ ]:
! pip install onednn

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=(3), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, verbose=1)

In [ ]:
# test model
test_data = {
    'fire_size': [37.0],
    'remoteness': [0.191923],
    'putout_time': [0]
}
test_data = pd.DataFrame(test_data)
test_data

In [ ]:
result = model.predict(test_data)
result

In [ ]:
# find class
final = 0
for i in range(6):
    if (result[0][i] > final):
        final = result[0][i]
    # print(final)
final = np.where(result[0] == final)
print(final[0][0])


In [ ]:
# save model
model.save("../Weights/cause.h5")

In [ ]:
# add cause to predicted fires dataset
predicted_data = pd.read_csv('../data/PredictedData.csv')

In [ ]:
predicted_data = predicted_data.dropna()
predicted_data['putout_time'] = predicted_data['putout_time'].str.split(' ').str[0]
test_data = predicted_data[['fire_size', 'remoteness', 'putout_time']].astype('float')
results = model.predict(test_data)

**XGBoost Classifier**

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
xgb_model = XGBClassifier(
        silent=False,
        scale_pos_weight=1,
        learning_rate=0.01,
        colsample_bytree = 1,
        subsample = 0.8,
        objective='multi:softprob',
        n_estimators=1000,
        reg_alpha = 0.3,
        max_depth=5,
        gamma=1)
xgb_model.fit(X_train, y_train)

In [ ]:
y_pred = xgb_model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
accuracy = round(accuracy_score(y_test, predictions), 2)
accuracy

In [ ]:
result = xgb_model.predict(test_data)
result[0]

In [ ]:
print(X_test.tail())
print(y_test.tail())

In [ ]:
# test model
test_data1 = {
    'fire_size': [5.0],
    'remoteness': [0.250806],
    'putout_time': [2.0]
}
test_data1 = pd.DataFrame(test_data1)
test_data1

In [ ]:
result1 = xgb_model.predict(test_data1)
result1[0].astype('int')